# Checkpointing ML models in PyTorch

This Notebook is based on Pytorch's basic example: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#saving-models

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [13]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [14]:
batch_size = 256

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [15]:
# Get cpu, gpu or mps device for training.
device = ("cpu")
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## Setting up Checkpointer
In its basic configuration, it is enough to simply save and load the model checkpoint. For a more useful application, the losses, epoch number and more can be stored defining a custom `checkpoint_function` and `restore_function`

In [19]:
from pathlib import Path
from checkpointer.checkpointer import Checkpointer

checkpointer = Checkpointer(
    local_checkpoint_file=Path("checkpoint.pt"),
    restore_function=lambda path: torch.load(path),
    checkpoint_function=lambda path, model: torch.save(model.state_dict(), path),
    checkpoint_every=100,
)

In [20]:
epochs = 2
state_dict = checkpointer.restore(model.state_dict())
model.load_state_dict(state_dict)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    checkpointer.checkpoint(model)

Epoch 1
-------------------------------
loss: 2.298964  [   64/60000]
loss: 2.290950  [ 6464/60000]
loss: 2.268433  [12864/60000]
loss: 2.267698  [19264/60000]
loss: 2.243402  [25664/60000]
loss: 2.216774  [32064/60000]
loss: 2.225729  [38464/60000]
loss: 2.187198  [44864/60000]
loss: 2.187707  [51264/60000]
loss: 2.162364  [57664/60000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.146602 

Epoch 2
-------------------------------
loss: 2.151340  [   64/60000]
loss: 2.150944  [ 6464/60000]
loss: 2.089247  [12864/60000]
loss: 2.116757  [19264/60000]
loss: 2.058106  [25664/60000]
loss: 1.993156  [32064/60000]
loss: 2.037151  [38464/60000]
loss: 1.943997  [44864/60000]
loss: 1.954421  [51264/60000]
loss: 1.898797  [57664/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.880005 



In [21]:
state_dict = checkpointer.restore(model.state_dict())
model.load_state_dict(state_dict)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    checkpointer.checkpoint(model)

Epoch 1
-------------------------------
loss: 1.903516  [   64/60000]
loss: 1.886605  [ 6464/60000]
loss: 1.760881  [12864/60000]
loss: 1.821035  [19264/60000]
loss: 1.701118  [25664/60000]
loss: 1.643944  [32064/60000]
loss: 1.692207  [38464/60000]
loss: 1.566296  [44864/60000]
loss: 1.601741  [51264/60000]
loss: 1.514306  [57664/60000]
Test Error: 
 Accuracy: 60.3%, Avg loss: 1.508909 

Epoch 2
-------------------------------
loss: 1.567466  [   64/60000]
loss: 1.540227  [ 6464/60000]
loss: 1.379034  [12864/60000]
loss: 1.471698  [19264/60000]
loss: 1.343177  [25664/60000]
loss: 1.330100  [32064/60000]
loss: 1.367873  [38464/60000]
loss: 1.265731  [44864/60000]
loss: 1.310724  [51264/60000]
loss: 1.225289  [57664/60000]
Test Error: 
 Accuracy: 63.5%, Avg loss: 1.235264 

